In [1]:
"""
Created on Wed Sep 26 11:31:14 2018

@author: yansel
"""
from __future__ import print_function
import numpy as np
import hbm
from vtkplotter import Plotter, Mesh, settings, Points, Lines

try:
    import cPickle as pickle
except:
    import pickle as pickle
try:
    import pymesh
except:
    pass
import json
import trimesh

In [2]:
#import trimesh
#import pyglet
SMPL_basicModel_f_lbs_path = "/media/neoglez/Data1/privat/PhD_Uni_Salzburg/DATASETS/smpl/models/basicModel_f_lbs_10_207_0_v1.0.0.pkl"
SMPL_basicModel_m_lbs_path = "/media/neoglez/Data1/privat/PhD_Uni_Salzburg/DATASETS/smpl/models/basicmodel_m_lbs_10_207_0_v1.0.0.pkl"

try:
    # Load pkl created in python 2.x with python 2.x
    female_model = pickle.load(open(SMPL_basicModel_f_lbs_path, 'rb'))
    male_model = pickle.load(open(SMPL_basicModel_m_lbs_path, 'rb'))
except:
    # Load pkl created in python 2.x with python 3.x
    female_model = pickle.load(open(SMPL_basicModel_f_lbs_path, 'rb'), encoding='latin1')
    male_model = pickle.load(open(SMPL_basicModel_m_lbs_path, 'rb'), encoding='latin1')
    
####################################################################
# Initialize the joints regressor as dense array (for clarity).    #
####################################################################

k_joints_predictor = female_model.get('J_regressor').A

new_female_joint_regressor = hbm.KJointPredictor(k_joints_predictor)

k_joints_predictor = male_model.get('J_regressor').A

new_male_joint_regressor = hbm.KJointPredictor(k_joints_predictor)

####################################################################
# Initialize the Osmpl female and male template.                   #
####################################################################
new_female_template = hbm.OSmplTemplate(female_model.get('v_template'),
                                    female_model.get('f'),
                                    female_model.get('weights'),
                                    female_model.get('shape_blend_shapes'),
                                    new_female_joint_regressor,
                                    male_model.get('kintree_table'),
                                    female_model.get('posedirs'))

new_male_template = hbm.OSmplTemplate(male_model.get('v_template'),
                                    male_model.get('f'),
                                    male_model.get('weights'),
                                    male_model.get('shape_blend_shapes'),
                                    new_male_joint_regressor, 
                                    male_model.get('kintree_table'),
                                    male_model.get('posedirs'))


In [3]:
####################################################################
# Once we have the template we instanciate the complete model.     #
####################################################################
human_female_model = hbm.OSmplWithPose(new_female_template,
                                   female_model.get('shapedirs').x,
                                   female_model.get('posedirs'), None, None)

human_male_model = hbm.OSmplWithPose(new_male_template,
                                 male_model.get('shapedirs').x,
                                 male_model.get('posedirs'), None, None)


# attach to logger so trimesh messages will be printed to console
#trimesh.util.attach_to_log()

# load a file by name or from a buffer
test_model = '/home/neoglez/H-DIM-Project/dataset/male/male_mesh_009041.obj'

human_male_model.read_from_obj_file(test_model)
########################################################################
# Test load annotation                                                 #
########################################################################


# load a file by name or from a buffer
test_annotation_file = '/home/neoglez/H-DIM-Project/dataset/annotations/male/male_mesh_anno_009041.json'
human_male_model.read_annotations_from_file(test_annotation_file)

# Initialize the pose vector. The pose vector has length 72 because it
# contains axis-angle representaions. Since the rig has 23 joints the vector
# has 23 x 3 + 3 (for the root joint) = 72 element. Note that the order of the
# elements is important: the first three elements correspond to the axis-angle
# representation of the first joint, and so forth.
pose = np.zeros(72)

human_male_model.apply_pose_blend_shapes(pose)

# perfom the skinning transformation
human_male_model.apply_linear_blend_skinning()

human = human_male_model.return_as_obj_format()
verts = np.array([line.split()[1:] for line in human.split('\n') 
               if line.startswith('v ')], dtype="f8")

faces = np.array([line.split()[1:] for line in human.split('\n') 
                  if line.startswith('f ')], dtype="int")
# remember that faces are 1-indexed in obj files therefore we have to
# one from every element.
faces = faces - 1

#  Loaded skeleton joints (total of 24) position of size (24, 3)
SkeletonJointsPosition = human_male_model.recompute_skeleton()
x_joints, y_joints, z_joints = [SkeletonJointsPosition[:, 0], 
                                SkeletonJointsPosition[:, 1],
                                SkeletonJointsPosition[:, 2]]

#  Loaded the kinamatic tree table of size (2, 24)
KinematicTreeTable = human_male_model.mean_template_shape.k_intree_table

settings.embedWindow(backend='k3d')
vp = Plotter()
m = Mesh([verts, faces], alpha=.1).flag('zero_pose').wireframe()
#m = vp.load(human)
m.color('blue')
vp += m

joints = Points(SkeletonJointsPosition, r=2).color('goldenrod')
vp += joints

In [4]:
###############################################################################
# Plot taking into consideration the joints tree                              #
###############################################################################
lines = []
for partnr, bone in enumerate(KinematicTreeTable.T):
    if partnr == 0:
        # This part starts "at infinity" and ends at the root joint
        continue
    line = [SkeletonJointsPosition[bone[0], :], 
            SkeletonJointsPosition[bone[1], :]]
    lines.append(line)

lines = Lines(np.array([line for line in lines])).color('black')
vp += lines

In [5]:
###############################################################################
# 'Clon' the human body and change pose to visualize both meshes #
###############################################################################
# Pose the model, in this case lower the arms 45 degrees.
pose_1_thetas = np.copy(pose)
pose_1_thetas[:] = np.random.rand(pose_1_thetas.size) * .2

human_male_model.apply_pose_blend_shapes(pose_1_thetas)

# perfom the skinning transformation
human_male_model.apply_linear_blend_skinning()

human = human_male_model.return_as_obj_format()
posed_verts = np.array([line.split()[1:] for line in human.split('\n') 
               if line.startswith('v ')], dtype="f8")

posedm = Mesh([posed_verts, faces], alpha=.1).flag('pose_1').wireframe()
posedm.color('green')
vp += posedm

PosedSkeletonJointsPosition = (
    human_male_model.return_posed_template_joint_locations())
posed_joints = Points(PosedSkeletonJointsPosition, r=2).color('darkmagenta')
vp += posed_joints

###############################################################################
# Plot taking into consideration the posed joints tree                        #
###############################################################################
lines = []
for partnr, bone in enumerate(KinematicTreeTable.T):
    if partnr == 0:
        # This part starts "at infinity" and ends at the root joint
        continue
    line = [PosedSkeletonJointsPosition[bone[0], :], 
            PosedSkeletonJointsPosition[bone[1], :]]
    lines.append(line)

posed_lines = Lines(np.array([line for line in lines])).color('saddlebrown')
vp += posed_lines


vp.show()

6
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
ia is mesh
<class 'vtkplotter.mesh.Mesh'>
ia is mesh
<class 'vtkplotter.shapes.Points'>
ia is mesh
<class 'vtkplotter.shapes.Lines'>
ia is mesh
<class 'vtkplotter.mesh.Mesh'>
ia is mesh
<class 'vtkplotter.shapes.Points'>
ia is mesh
<class 'vtkplotter.shapes.Lines'>


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[0.0265926358763537…